In [1]:
import urllib.request 
import re
import pandas as pd
from elasticsearch import Elasticsearch

In [2]:
import requests
# run bin/elasticsearch in elasticsearch directory in terminal before running this cell
res = requests.get('http://localhost:9200')

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [4]:
def test_ES(es):
    return es.ping()  # got True

In [5]:
if test_ES(es):
    print('ES instance working')
else:
    print('ES instance not working')

ES instance working


/home/mark/anaconda3/envs/nlp/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [6]:
def index_info(index_name):
    count, deleted, shards, =  es.cat.indices(index=index_name, h=['docs.count', 'docs.deleted', 'pri'])[:-1].split(' ')
    print(
      """
      #### INDEX INFO #####
      index_name = {}
      doc_count = {}
      shard_count = {}
      deleted_doc_count = {}
      """.format(index_name, count, shards, deleted)
  )

In [7]:
# read data from steam data directory 
games = pd.read_pickle('final_data_raw.pickle')

In [8]:
games['release_date'] = pd.to_datetime(games.release_date)

In [9]:
#es.indices.delete(index='test-index')

In [10]:
# set index name
index_name = 'steam_index_bm25'

In [11]:
# BM25 (Default index)
request_body_bm25 = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 1,
        
    },
    'mappings': {
          'properties': {
              'app_id':{'type': 'integer'},
              'title': {'type': 'text'},
              'short_description': {'type': 'text',
                                    'analyzer': 'english'},
              'about_the_game': {'type': 'text',
                                 'analyzer': 'english'},
              'developers': {
                    "type": "text",
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'publishers': {
                    "type": "text",
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'categories': {
                    "type": "text",
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'genres': {
                    "type": "text",
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'release_date': {'type': 'date',
                              },
          }
    }
}

try:
    es.indices.get(index_name)
    print('index {} already exists'.format(index_name))
except:
    print('creating index {}'.format(index_name))
    es.indices.create(index_name, body=request_body_bm25)

index steam_index_bm25 already exists


/home/mark/anaconda3/envs/nlp/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [12]:
for i, (name, appid, about_the_game, short_description,
       developers, publishers, categories, genres, release_date) in enumerate(games.values[:,1:]):
    doc_body = {
              'app_id': app_id,
              'title': name,
              'short_description': short_description,
              'about_the_game': about_the_game,
              'developers': developers,
              'publishers': publishers,
              'categories': categories,
              'genres': genres,
              'release_date': release_date
              }
    es.index(index_name, doc_body, id=i)'''

"for i, (name, appid, about_the_game, short_description,\n       #developers, publishers, categories, genres, release_date) in enumerate(games.values[:,1:]):\n    doc_body = {\n              'app_id': app_id,\n              'title': name,\n              'short_description': short_description,\n              'about_the_game': about_the_game,\n              'developers': developers,\n              'publishers': publishers,\n              'categories': categories,\n              'genres': genres,\n              'release_date': release_date\n              }\n    es.index(index_name, doc_body, id=i)"

In [13]:
print('we have made and index called {} with {} documents'.format(index_name, es.cat.count(index=index_name,h=['count'])))
index_info(index_name)

we have made and index called steam_index_bm25 with 49317
 documents

      #### INDEX INFO #####
      index_name = steam_index_bm25
      doc_count = 49317
      shard_count = 1
      deleted_doc_count = 0
      


In [37]:
index_name = 'steam_index_dfr'

In [102]:
#DFR index
request_body_dfr = {
        'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 1,
        'index': {
            'similarity': {
                'dfr_similarity': {
                    'type': 'DFR',
                    'basic_model': 'g',
                    'after_effect': 'l',
                    'normalization': 'h2',
                    'normalization.h2.c':'3.0'

                }
            }
        }
        
    },
    'mappings': {
          'properties': {
              'app_id':{'type': 'integer'},
              'title': {'type': 'text', 'similarity': 'dfr_similarity'},
              'short_description': {'type': 'text', 'analyzer': 'english', 'similarity': 'dfr_similarity'},
              'about_the_game': {'type': 'text', 'analyzer': 'english', 'similarity': 'dfr_similarity'},
              'developer': {
                    "type": "text", 'similarity': 'dfr_similarity',
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'publisher': {
                    "type": "text", 'similarity': 'dfr_similarity',
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'categories': {
                    "type": "text", 'similarity': 'dfr_similarity',
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'genres': {
                    "type": "text", 'similarity': 'dfr_similarity',
                    "fields": {
                    "raw": { 
                       "type": "keyword"
                            }
                    }
              },
              'release_date': {'type': 'date'},
          }
    }
}

try:
    es.indices.get(index_name)
    print('index {} already exists'.format(index_name))
except:
    print('creating index {}'.format(index_name))
    es.indices.create(index_name, body=request_body_dfr)

index steam_index_dfr already exists


In [ ]:
for i, (title, app_id, developer, publisher, categories, genres, body, abstract, release_date) in enumerate(games.values[:,1:-1]):
    doc_body = {
              'app_id': app_id,
              'title': title,
              'short_description': abstract,
              'about_the_game': body,
              'developer': developer,
              'publisher': publisher,
              'categories': categories,
              'genres': genres,
              'release_date': release_date
              }
    es.index(index_name, doc_body, id=i)

In [103]:
print('we have made and index called {} with {} documents'.format(index_name, es.cat.count(index=index_name,h=['count'])))
index_info(index_name)

we have made and index called steam_index_dfr with 49317
 documents

      #### INDEX INFO #####
      index_name = steam_index_dfr
      doc_count = 49317
      shard_count = 1
      deleted_doc_count = 0
      


In [54]:
def search():
    query = text.value
    fields = ["title", "short_description", 'about_the_game', 'developer', 'categories', 'genres']

    start_date = date_slider.result[0]
    end_date = date_slider.result[1]
    
    genre = genres_drop.value
    category = categories_drop.value

    query_body = {
        "query": {
            "bool": {
                "should": [
                    {
                        "multi_match": {
                            "query": query, 
                            "fields":  fields
                        }
                    }
                ],
                "must": [
                    {
                        'range': {
                            'release_date': {
                                'gte': start_date,
                                'lte': end_date
                            }
                        }
                    }
                ],
                 'filter':[
                    {'wildcard': {'genres.raw': genre}},
                    {'wildcard': {'categories.raw': category}}
                    ]
                }
            }
        }

    print('### SEARCH RESULTS ####\n')
    results = es.search(index=index_name, body=query_body)['hits']['hits']
    for hit in results:
        print('GAME TITLE: {}, Release date: {}, Score: {}\n\n{}\n\nDeveloper(s): {}\nPublisher(s): {}\n\nGenre(s): {}\nCategories: {}\n'.format(hit['_source']['title'], 
                                                                                          hit['_source']['release_date'][:10], 
                                                                                          hit['_score'], hit['_source']['about_the_game'],
                                                                                          hit['_source']['developers'], hit['_source']['publishers'],
                                                                                          hit['_source']['genres'], hit['_source']['categories']))

In [14]:
genre_list = []
for genres in games.genres:
    for g in genres:
        genre_list.append(g)
        
genre_list = list(set(genre_list)) + ['*']   

category_list = []
for categories in games.categories:
    for c in categories:
        category_list.append(c)
        
category_list = list(set(category_list)) + ['*'] 

In [29]:
import ipywidgets as widgets
from datetime import datetime
from ipywidgets import interactive
from IPython.display import display, clear_output

dates = pd.date_range(games.release_date.min(), games.release_date.max())

options = [(date.strftime(' %d\%m\%Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Release date',
    orientation='horizontal',
    layout={'width': '500px'}
)

# Define any function
def return_dates(date_range):
    return date_range

# Create sliders using interactive
date_slider = interactive(return_dates, date_range = selection_range_slider)

text = widgets.Text(
       placeholder = 'Enter query',
       description='Search',
        layout={'width': '500px'})

button = widgets.Button(description='Search')
out = widgets.Output()

def on_button_clicked(_):
      # "linking function with output"
    with out:
      # what happens when we press the button
        clear_output()
        search()
    
# linking button and function together using a button's method
button.on_click(on_button_clicked)

genres_drop = widgets.Dropdown(description = 'Genre',
                options = genre_list, value = '*')

categories_drop = widgets.Dropdown(description = 'Catagory',
                                  options = category_list, value = '*')

# You can also view this in a notebook without using display.
display(text, date_slider, genres_drop, categories_drop, button, out)

Text(value='', description='Search', layout=Layout(width='500px'), placeholder='Enter query')

interactive(children=(SelectionRangeSlider(description='Release date', index=(0, 15711), layout=Layout(width='…

Dropdown(description='Genre', index=35, options=('Episodic', 'Documentary', 'Free to Play', 'Early Access', 'S…

Dropdown(description='Catagory', index=37, options=('Co-op', 'Shared/Split Screen Co-op', 'Online PvP', 'Comme…

Button(description='Search', style=ButtonStyle())

Output()